# BOW 기반의 카운트 벡터 생성

NLTK가 제공하는 영화 리뷰

In [ ]:
import nltk
nltk.download('movie_reviews')

`movie_reviews`의 주요 함수

In [ ]:
from nltk.corpus import movie_reviews

print('review count:', len(movie_reviews.fileids()))   # 영화 리뷰 문서의 id를 반환
print('samples of file ids:', movie_reviews.fileids()[:10])    # id를 10개까지만 출력
print('categories of reviews:', movie_reviews.categories())    # label, 즉 긍정인지 부정인지에 대한 분류
print('num of "neg" reviews:', len(movie_reviews.fileids(categories='neg')))   # label이 부정인 문서들의 id를 반환
print('num of "pos" reviews:', len(movie_reviews.fileids(categories='pos')))   # label이 긍정인 문서들의 id를 반환

# 첫 번째 문서
fileid = movie_reviews.fileids()[0]
print('id of the first review:', fileid)
print('first review content:\n', movie_reviews.raw(fileid)[:200])  # 첫 번째 문서의 내용을 200자까지만 출력
print()
print('sentence tokenization result:', movie_reviews.sents(fileid)[:2])    # 첫 번째 문서를 sentence tokenize한 결과 중 앞 두 문장
print('word tokenization result:', movie_reviews.words(fileid)[:20])   # 첫 번째 문서를 word tokenize한 결과 중 앞 스무 단어

review count: 2000
samples of file ids: ['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']
categories of reviews: ['neg', 'pos']
num of "neg" reviews: 1000
num of "pos" reviews: 1000
id of the first review: neg/cv000_29416.txt
first review content:
 plot : two teen couples go to a church party , drink and then drive . 
they get into an accident . 
one of the guys dies , but his girlfriend continues to see him in her life , and has nightmares . 
w

sentence tokenization result: [['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.']]
word tokenization result: ['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'a

In [ ]:
# 첫 번째 문서의 앞 50개 단어를 출력

documents = [list(movie_reviews.words(fileid)) for fileid in movie_reviews.fileids()]
print(documents[0][:50])

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.', 'they', 'get', 'into', 'an', 'accident', '.', 'one', 'of', 'the', 'guys', 'dies', ',', 'but', 'his', 'girlfriend', 'continues', 'to', 'see', 'him', 'in', 'her', 'life', ',', 'and', 'has', 'nightmares', '.', 'what', "'", 's', 'the', 'deal', '?', 'watch']


**정규표현식 토큰화 & 불용어 제거**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer("[\w']{3,}")    # 정규포현식으로 토크나이저를 정의
english_stops = set(stopwords.words('english'))     # 영어 불용어사전을 가져옴

# words() 대신 raw()를 이용해 원문을 가져옴
documents = [movie_reviews.raw(fileid) for fileid in movie_reviews.fileids()] 

# stopwords의 적용과 토큰화를 동시에 수행
tokens = [[token for token in tokenizer.tokenize(doc) if token not in english_stops] for doc in documents]

# 단어의 빈도 수 카운트
word_count = {}
for text in tokens:
    for word in text:
        word_count[word] = word_count.get(word, 0) + 1

sorted_features = sorted(word_count, key=word_count.get, reverse=True)  # 내림차순 정렬

print('num of features:', len(sorted_features))
for word in sorted_features[:10]:
    print(f"count of '{word}': {word_count[word]}", end=', ')

num of features: 43030
count of 'film': 8935, count of 'one': 5791, count of 'movie': 5538, count of 'like': 3690, count of 'even': 2564, count of 'time': 2409, count of 'good': 2407, count of 'story': 2136, count of 'would': 2084, count of 'much': 2049, 

**특성 집합 생성**

In [ ]:
word_features = sorted_features[:1000] # 빈도가 높은 상위 1000개의 단어만 추출하여 features를 구성

**카운트 벡터 생성**

In [ ]:
def document_features(document, word_features):
    word_count = {}
    for word in document:   # document에 있는 단어들에 대해 빈도수를 먼저 계산
        word_count[word] = word_count.get(word, 0) + 1
        
    features = []
    for word in word_features:      # word_features의 단어에 대해 계산된 빈도수를 feature에 추가
        features.append(word_count.get(word, 0))    # 빈도가 없는 단어는 0을 입력
    return features

In [ ]:
feature_sets = [document_features(d, word_features) for d in tokens]

# 첫째 feature set의 내용을 앞 20개만 word_features의 단어와 함께 출력
for i in range(20):
    print(f'({word_features[i]}, {feature_sets[0][i]})', end=', ')

(film, 5), (one, 3), (movie, 6), (like, 3), (even, 3), (time, 0), (good, 2), (story, 0), (would, 1), (much, 0), (also, 1), (get, 3), (character, 1), (two, 2), (well, 1), (first, 0), (characters, 1), (see, 2), (way, 3), (make, 5), 

# 사이킷런을 이용한 카운트 벡터 생성

# 카운트 벡터의 활용

코사인 유사도(Cosine similarity)

# TF-IDF로 성능향상